In [268]:
import pandas as pd 
pd.set_option('max_colwidth', None)
pd.options.display.max_rows = 10000
import re
import html
from numpy import NaN
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import numpy as np 
from collections import Counter

from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from heapq import nlargest

nlp = spacy.load("en_core_web_sm")
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('brown')
nltk.download('vader_lexicon')

import contractions
import torch






[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/parthmalik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/parthmalik/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/parthmalik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /Users/parthmalik/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/parthmalik/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [215]:
# LOAD + DATE SET + SORT ASC BY DATE
file = './TweetsFinalFiltered.csv'
df = pd.read_csv(file, low_memory=False)
df['created_at'] = pd.to_datetime(df['created_at'], format='%Y-%m-%d')
df = df.sort_values('created_at')

In [216]:
from ff_dict import merge
sf_ff_dict = merge()
sf_ff_dict['TA'] = 'Technical Analysis'

In [217]:
df.head()

,created_at,entities,like_count,quote_count,reply_count,retweet_count,full_text,coin_id,datasource
0,2021-02-01,NaN,154,NaN,18,23,#privacy is a human right. learn how to make your #bitcoin transactions more private in this clip: https://t.co/fnadsxffcu https://t.co/nznajw8g2m,btc,influtweets
1,2021-02-01,NaN,65,NaN,8,13,"attended any of the 30+ livestream events iâve held over the last year? together, weâve learned a lot about #bitcoin #ethereum &amp; #lightningnetwork. starting this month iâll be moving them to sundays. subscribe to be notified as they're announced: https://t.co/rgoqfv22mr https://t.co/ec4ugzrilq",amp,influtweets
2,2021-02-01,NaN,65,NaN,8,13,"attended any of the 30+ livestream events iâve held over the last year? together, weâve learned a lot about #bitcoin #ethereum &amp; #lightningnetwork. starting this month iâll be moving them to sundays. subscribe to be notified as they're announced: https://t.co/rgoqfv22mr https://t.co/ec4ugzrilq",btc,influtweets
3,2021-02-01,NaN,65,NaN,8,13,"attended any of the 30+ livestream events iâve held over the last year? together, weâve learned a lot about #bitcoin #ethereum &amp; #lightningnetwork. starting this month iâll be moving them to sundays. subscribe to be notified as they're announced: https://t.co/rgoqfv22mr https://t.co/ec4ugzrilq",eth,influtweets
4,2021-02-01,"{'cashtags': [{'start': 19, 'end': 23, 'tag': 'STC'}, {'start': 28, 'end': 33, 'tag': 'WELL'}]}",1,1.0,1,1,Own shares in both $STC and $WELL,stc,othertweets


In [218]:
df.tail()

,created_at,entities,like_count,quote_count,reply_count,retweet_count,full_text,coin_id,datasource
272253,2023-06-12,NaN,0,NaN,0,8,rt @crypto_crib_: ð²chinese bank boci issues country's first tokenized security on ethereum blockchain. https://t.co/yeoudbbqzc,eth,influtweets
272252,2023-06-12,NaN,56,NaN,16,7,"bitcoin, not crypto.\n\ncrypto, not security.",btc,influtweets
272251,2023-06-12,NaN,1,NaN,9,0,new gap cme - 26400 ð\n\nno better time to join...\n60% offð¥ð¥ð¥\nclickð\nhttps://t.co/tcetr8tf7t\n\n$btc $eth $lina $xrp $arb $bnb $ltc $key $amb $sui\n$amb $bts $nkn $syn $ldo $combo $qkc $rdnt $mdt $inj https://t.co/qaryyzrfuq,xrp,influtweets
272258,2023-06-12,NaN,3,NaN,5,0,"booomð¥\n\nour #ai bot/indicator crushes another $lina trade perfectly! ð¾ð°\n\n#lina tp #9â \nperiod: 1 day 26 minâ°\nprofit: +1,391.19%ð´\n\n(2) $99 lifetime memberships left ð¥\npay via usdt (trc20)ð \ntzfsdngc7n5gvgxsbfgc4mivqpjymfde6t\n\n#doge #apt #kas #hbar #ftm #inj #matic $pepe https://t.co/4qqsjur2dp",usdt,influtweets
272460,2023-06-12,NaN,0,NaN,0,8,"rt @crypto_crib_: jpmorgan calls for ""comprehensive framework"" after sec's lawsuit on binance &amp; coinbase; urges law-makers to adopt sec staâ¦",amp,influtweets


**1. Frequency Analysis of Words and Phrases**

In [177]:
texts = df['full_text']

# Tokenization
tokens = [token for text in texts for token in text.split()]

# Single words frequency
word_freq = Counter(tokens)

# Bigrams frequency
bigrams = Counter(ngrams(tokens, 2))

print(word_freq.most_common(10))
print(bigrams.most_common(10))

[('the', 135531), ('to', 100692), ('and', 81083), ('on', 72263), ('a', 59836), ('is', 57952), ('in', 57121), ('for', 53357), ('of', 45271), ('$ETH', 43348)]
[(('to', 'the'), 11342), (('in', 'the'), 11040), (('on', 'the'), 10531), (('$BTC', '$ETH'), 10015), (('will', 'be'), 8892), (('for', 'the'), 8884), (('#safemoon', '#safemars'), 7997), (('is', 'a'), 7298), (('of', 'the'), 6715), (('$btc', '$eth'), 6489)]


**2. Analysis of Special Characters and Non-Standard Text**

In [178]:
special_chars = Counter()

for text in texts:
    special_chars.update(Counter(re.findall(r'[^a-zA-Z0-9\s]', text)))

print(special_chars.most_common(500))

[('$', 1407434), ('#', 954730), ('/', 672667), ('.', 526549), ('@', 366772), (':', 324501), (',', 158908), ('!', 151180), ('🚀', 86136), ('-', 78267), ('_', 71245), ('%', 67332), ("'", 40816), (')', 36585), ('🔥', 36193), ('(', 34220), ('️', 32537), ('\x9f', 31973), (';', 31910), ('ð', 31138), ('&', 26389), ('?', 23838), ('✅', 23216), ('â', 20239), ('+', 19117), ('’', 14440), ('💎', 11689), ('"', 11090), ('\x80', 10670), ('💰', 9848), ('🎉', 9741), ('\x8f', 9631), ('\x92', 9142), ('¸', 8653), ('ï', 8075), ('🟢', 7316), ('£', 7101), ('¥', 6599), ('\x83', 6466), ('👉', 6403), ('\x91', 5887), ('|', 5869), ('*', 5866), ('🤑', 5379), ('🟡', 4928), ('👑', 4892), ('\x87', 4702), ('=', 4517), ('🥞', 4273), ('👇', 4186), ('💵', 4166), ('📈', 4099), ('💥', 4023), ('➡', 4008), ('[', 3846), (']', 3841), ('❤', 3555), ('⇝', 3510), ('¦', 3473), ('⚡', 3372), ('🏻', 3360), ('\x99', 3358), ('\x9c', 3304), ('👌', 3247), ('\x94', 3211), ('💯', 3206), ('📥', 3110), ('☁', 3023), ('°', 3004), ('\x90', 2992), ('🚨', 2964), ('😍',

**3. Hashtag and Mention Analysis**

In [164]:
hashtags = Counter()
mentions = Counter()

for text in texts:
    hashtags.update(Counter(re.findall(r'#\w+', text)))
    mentions.update(Counter(re.findall(r'@\w+', text)))

print(hashtags.most_common(10))
print(mentions.most_common(10))

[('#safemars', 34710), ('#SAFEMARS', 32582), ('#BSC', 26541), ('#safemoon', 19919), ('#BNB', 18402), ('#SAFEMOON', 16897), ('#bitcoin', 16580), ('#dogecoin', 14931), ('#doge', 14586), ('#btc', 13601)]
[('@Safemartians', 12030), ('@elonmusk', 7673), ('@binance', 4960), ('@Royale_Finance', 4897), ('@cz_binance', 4501), ('@PancakeSwap', 4270), ('@CoinMarketCap', 3696), ('@ALTSTEINN', 3225), ('@DEG_APE', 2805), ('@CROSSSWAPDEX', 2728)]


**4. URL Analysis**

In [179]:
url_count = sum(bool(re.search(r'https?://\S+|www\.\S+', text)) for text in texts)
print(f"Number of tweets with URLs: {url_count}")


Number of tweets with URLs: 168501


**5. Length and Readability Analysis**

In [180]:
word_counts = [len(text.split()) for text in texts]
print(f"Average word count: {sum(word_counts) / len(word_counts)}")

Average word count: 27.571971768436583


**6. Contextual Analysis of Crypto Terms**

In [181]:
def abbreviations_match_check_3(crypto_dict, texts, max_samples=20):
    match_words_in_full_text = {}
    non_match_words = {}
    sample_texts = {}

    for term in crypto_dict.keys():
        term_pattern = r'\b' + re.escape(term) + r'\b'  # Matches the term as a whole word
        term_count = 0
        samples = []

        for text in texts:
            if re.search(term_pattern, text):
                term_count += 1
                if len(samples) < max_samples:
                    samples.append(text)

        if term_count == 0:
            non_match_words[term] = term_count
        else:
            match_words_in_full_text[term] = term_count
            sample_texts[term] = samples

    match_words_in_full_text = dict(sorted(match_words_in_full_text.items(), key=lambda x: x[1], reverse=True))
    non_match_words = dict(sorted(non_match_words.items(), key=lambda x: x[1], reverse=True))

    return match_words_in_full_text, non_match_words, sample_texts

# Example usage
matched_abbreviations_frequency_dict, non_matched_abbreviations_frequency_dict, sample_texts = abbreviations_match_check_3(sf_ff_dict, texts)

# Getting top match words with their samples
top_match_words = []
for key in nlargest(len(matched_abbreviations_frequency_dict), matched_abbreviations_frequency_dict, key=matched_abbreviations_frequency_dict.get):
    if matched_abbreviations_frequency_dict[key] >= 20:
        top_match_words.append((key, matched_abbreviations_frequency_dict[key], sample_texts[key]))

In [182]:
# c = 1
# for key, value in matched_abbreviations_frequency_dict.items():
#     print(f'({c})Key : {key}\nValue : {value}\n')
#     c+=1
print(len(sf_ff_dict))
print(len(matched_abbreviations_frequency_dict))
print(len(non_matched_abbreviations_frequency_dict))

81
64
17


In [199]:
matched_abbreviations_frequency_dict.items()

dict_items([('NFT', 19368), ('DeFi', 13005), ('DEX', 4592), ('DYOR', 3404), ('HODL', 3069), ('ATH', 2339), ('AMA', 1662), ('FOMO', 1298), ('CMC', 1069), ('SC', 831), ('APY', 614), ('APR', 600), ('ICO', 573), ('MCAP', 439), ('TA', 416), ('DAO', 358), ('KYC', 345), ('SEC', 299), ('CT', 220), ('FUD', 219), ('ITO', 199), ('Altcoin', 181), ('CEX', 142), ('DLT', 134), ('EIP', 130), ('MACD', 125), ('DCA', 91), ('FA', 73), ('ERC', 68), ('ALT', 53), ('BUIDL', 52), ('EVM', 51), ('ERC-20', 47), ('ETF', 47), ('P2P', 47), ('BTFD', 45), ('Lambo', 44), ('REKT', 43), ('YTD', 43), ('DApp', 39), ('PoS', 39), ('OTC', 35), ('PoW', 27), ('CFTC', 16), ('TLT', 15), ('Tx', 14), ('TPS', 11), ('FIAT', 7), ('SATS', 7), ('ATL', 6), ('BFT', 6), ('LN', 6), ('STO', 6), ('FTC', 4), ('ERC-721', 3), ('AML', 2), ('PnD', 2), ('SoV', 2), ('TOR', 2), ('BIP', 1), ('BTD', 1), ('MoE', 1), ('PoB', 1), ('mBTC', 1)])

In [200]:
non_matched_abbreviations_frequency_dict.items()

dict_items([('DAICO', 0), ('DOJ', 0), ('DPoS', 0), ('EEA', 0), ('ETP', 0), ('FDIC', 0), ('FUDster', 0), ('IBO', 0), ('JOMO', 0), ('OCO', 0), ('PoA', 0), ('PoD', 0), ('SegWit', 0), ('TxID', 0), ('UXTO', 0), ('UoA', 0), ('uBTC', 0)])

**NLP STUFF HERE**

**NLP STUFF HERE**

In [256]:
# # Preprocessing function
CUSTOM_STOPWORDS = set(stopwords.words('english')) - set(["not", "no", "down", "up", "above", "below", "against", "now"])
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):

    crypto_dict = sf_ff_dict

    # Decode HTML entities
    text = html.unescape(text)

    # Expand contractions in the text
    text = contractions.fix(text)

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove user mentions
    text = re.sub(r'@\w+', '', text)

    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Remove newline and unicode characters like \u200b
    text = re.sub(r'[\r\n\u200b]+', ' ', text)

    # Replace terms based on the crypto dictionary
    for term, full_form in crypto_dict.items():
        term_pattern = r'\b' + re.escape(term) + r'\b'  # Matches the term as a whole word
        text = re.sub(term_pattern, full_form, text)

    # Remove punctuation (optional: keep some punctuation)
    text = re.sub(r'[^\w\s$%]', '', text)

    # Handle negations
    text = re.sub(r'\bnot\b|\bno\b', 'not_', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenization and lemmatization
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in CUSTOM_STOPWORDS and len(token) > 1]

    # Rejoin tokens into a single string
    clean_text = ' '.join(tokens)
    return clean_text

**APPLYING NLP PREPROCESSING FN HERE**

In [220]:
def safe_preprocess(text):
    try:
        return preprocess_text(text)
    except IndexError as e:
        print(f"Error with text: {text}")
        # Return the original text or some placeholder to indicate a problem
        return None

In [221]:
df['clean_text'] = df['full_text'].apply(safe_preprocess)

Error with text: $PPAY 20 + DEX integrations are coming in April.🔥

✅Polkadot
✅YFI
✅BinanceChain
✅PancakeSwap 
✅ZERO GAS
✅İDO coming...

$SXP $DAO $YOP $CSPR #BSC #BTC #BNB #TRX $XRP $DUCK $SWAP $POLS $ZEE $DOT $CAKE $BAKE #BTT $HT $OKB $KCS $LINK $ONT $HOT $GT $NEO #doge #bitci #dogecoin https://t.co/rDiqzy11Ur
Error with text: @binance @MyNeighborAlice SAFE MARS ARMY!!! 🚀🚀🚀🚀🔥🔥🔥🔥💎💎💎💎💎 @Safemartians @SafemarsTR @cz_binance @elonmusk  #SAFEMARS #SafeMars2Mars HADİ FAVLAYIN !!!!! https://t.co/14fmwQCS2u
Error with text: 👀Don’t disregard #MoonStar

👥Community is EVERYTHING in crypto

📈Our community is 50,000+ hodlers!

$akita $shiba $doge $btc $eth $crypto #safebtc #safemoon #safemars #BONFIRE #safegalaxy #Rmoon #CAROM #elongate #doge #BSC #moonshot #BTC #ETH #NFT #DEFİ #ALTSEASON https://t.co/ljG3CdzvhU
Error with text: Hello, Buy HamsiToken now at Pancakeswap, don't miss the rise 🚀🚀🚀🚀🚀🚀🚀
visit our website :https://t.co/u6ozPnxSLi✅
#HAMSİ #hamsi #Trabzonspor #trabzon #safemoon #VanCat #d

In [222]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272461 entries, 0 to 272460
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   created_at     272461 non-null  datetime64[ns]
 1   entities       245708 non-null  object        
 2   like_count     272461 non-null  int64         
 3   quote_count    245708 non-null  float64       
 4   reply_count    272461 non-null  int64         
 5   retweet_count  272461 non-null  int64         
 6   full_text      272461 non-null  object        
 7   coin_id        272461 non-null  object        
 8   datasource     272461 non-null  object        
 9   clean_text     272444 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 22.9+ MB


**DROPPING THE ERROR ROWS AS THEY ARE MOSTLY USE MENTIONS AND HAVE NO SENTIMENTS**

In [223]:
df = df.dropna(subset=['clean_text'])

In [239]:
# Rearranging Cols
df = df.reindex(columns=['created_at', 'entities', 'like_count', 'quote_count', 'reply_count',
       'retweet_count', 'full_text', 'clean_text',  'coin_id', 'datasource'])

In [ ]:
df[['full_text', 'clean_text']].head(15)

In [224]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272444 entries, 0 to 272460
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   created_at     272444 non-null  datetime64[ns]
 1   entities       245691 non-null  object        
 2   like_count     272444 non-null  int64         
 3   quote_count    245691 non-null  float64       
 4   reply_count    272444 non-null  int64         
 5   retweet_count  272444 non-null  int64         
 6   full_text      272444 non-null  object        
 7   coin_id        272444 non-null  object        
 8   datasource     272444 non-null  object        
 9   clean_text     272444 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 22.9+ MB


In [241]:
text_pairs = list(zip(df['full_text'], df['clean_text']))

In [277]:
len(text_pairs)

272444

In [262]:
text_pairs[len(text_pairs)//2: len(text_pairs)//2 + 16]

[('Thanks so much Elon Musk!!\n\nSent 5 #eth got back 50! Highly recommended. #ethereum\n\n#eth $eth #dogecoin #ethereum $doge #bitcoin $btc #btc #bnb #binance #coinbase\nhttps://t.co/4vUM2FbbaW',
  'thanks much elon musk sent eth got back 50 highly recommended ethereum eth eth dogecoin ethereum doge bitcoin btc btc bnb binance coinbase'),
 ('@BabyCakeBSC @YouTube #BabyCake #babycakeBSC #cake #pancakeswap #BSCGems \n#BSCNews #CryptoNews #Crypto #DeFi #cz_binance\n@babycakeBSC @pancakeswap $CAKE',
  'babycakebsc youtube babycake babycakebsc cake pancakeswap bscgems bscnews cryptonews crypto decentralized finance cz_binance babycakebsc pancakeswap cake'),
 ('$ETH to 10k', 'eth 10k'),
 ("@tyler I'm long term bullish on $ETH and $CVP and If BSC or Binance supports BSCDEFI portfolio by @PowerPoolCVP  $CVP is bound to moon",
  'tyler long term bullish eth cvp bsc binance support bscdefi portfolio powerpoolcvp cvp bound moon'),
 ('🚨Inaugural NUMI Sets 🚨 ⤵️ \n\n1⃣ The OGs: 1 $BTC &amp; 1 $ETH 

In [278]:
len(text_pairs) == len(df_2)

True

In [253]:
df_2 = df


<class 'pandas.core.frame.DataFrame'>
Index: 272444 entries, 0 to 272460
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   created_at     272444 non-null  datetime64[ns]
 1   entities       245691 non-null  object        
 2   like_count     272444 non-null  int64         
 3   quote_count    245691 non-null  float64       
 4   reply_count    272444 non-null  int64         
 5   retweet_count  272444 non-null  int64         
 6   full_text      272444 non-null  object        
 7   clean_text     272444 non-null  object        
 8   coin_id        272444 non-null  object        
 9   datasource     272444 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 22.9+ MB


In [254]:
df_2 = df_2.drop('clean_text', axis=1)

In [259]:
df_2.tail()

,created_at,entities,like_count,quote_count,reply_count,retweet_count,full_text,coin_id,datasource,clean_text
272253,2023-06-12,NaN,0,NaN,0,8,rt @crypto_crib_: ð²chinese bank boci issues country's first tokenized security on ethereum blockchain. https://t.co/yeoudbbqzc,eth,influtweets,rt chinese bank boci issue country first tokenized security ethereum blockchain
272252,2023-06-12,NaN,56,NaN,16,7,"bitcoin, not crypto.\n\ncrypto, not security.",btc,influtweets,bitcoin not_ crypto crypto not_ security
272251,2023-06-12,NaN,1,NaN,9,0,new gap cme - 26400 ð\n\nno better time to join...\n60% offð¥ð¥ð¥\nclickð\nhttps://t.co/tcetr8tf7t\n\n$btc $eth $lina $xrp $arb $bnb $ltc $key $amb $sui\n$amb $bts $nkn $syn $ldo $combo $qkc $rdnt $mdt $inj https://t.co/qaryyzrfuq,xrp,influtweets,new gap cme 26400 not_ better time join 60 click btc eth lina xrp arb bnb ltc key amb sui amb bts nkn syn ldo combo qkc rdnt mdt inj
272258,2023-06-12,NaN,3,NaN,5,0,"booomð¥\n\nour #ai bot/indicator crushes another $lina trade perfectly! ð¾ð°\n\n#lina tp #9â \nperiod: 1 day 26 minâ°\nprofit: +1,391.19%ð´\n\n(2) $99 lifetime memberships left ð¥\npay via usdt (trc20)ð \ntzfsdngc7n5gvgxsbfgc4mivqpjymfde6t\n\n#doge #apt #kas #hbar #ftm #inj #matic $pepe https://t.co/4qqsjur2dp",usdt,influtweets,booom ai botindicator crush another lina trade perfectly lina tp period day 26 min profit 139119 99 lifetime membership left pay via usdt trc20 tzfsdngc7n5gvgxsbfgc4mivqpjymfde6t doge apt ka hbar ftm inj matic pepe
272460,2023-06-12,NaN,0,NaN,0,8,"rt @crypto_crib_: jpmorgan calls for ""comprehensive framework"" after sec's lawsuit on binance &amp; coinbase; urges law-makers to adopt sec staâ¦",amp,influtweets,rt jpmorgan call comprehensive framework sec lawsuit binance coinbase urge lawmaker adopt sec sta


In [257]:
df_2['clean_text'] = df_2['full_text'].apply(safe_preprocess)

In [ ]:
df_2 = df_2.drop('entities', axis=1)

In [266]:
df_2.iloc[len(text_pairs)//2: len(text_pairs)//2 + 16]

,created_at,like_count,quote_count,reply_count,retweet_count,full_text,coin_id,datasource,clean_text
145292,2021-08-06,0,0.0,0,0,Thanks so much Elon Musk!!\n\nSent 5 #eth got back 50! Highly recommended. #ethereum\n\n#eth $eth #dogecoin #ethereum $doge #bitcoin $btc #btc #bnb #binance #coinbase\nhttps://t.co/4vUM2FbbaW,eth,othertweets,thanks much elon musk sent eth got back 50 highly recommended ethereum eth eth dogecoin ethereum doge bitcoin btc btc bnb binance coinbase
146856,2021-08-06,0,0.0,0,0,@BabyCakeBSC @YouTube #BabyCake #babycakeBSC #cake #pancakeswap #BSCGems \n#BSCNews #CryptoNews #Crypto #DeFi #cz_binance\n@babycakeBSC @pancakeswap $CAKE,cake,othertweets,babycake babycakebsc cake pancakeswap bscgems bscnews cryptonews crypto decentralized finance cz_binance cake
145583,2021-08-06,0,0.0,0,0,$ETH to 10k,eth,othertweets,eth 10k
145585,2021-08-06,0,0.0,0,0,@tyler I'm long term bullish on $ETH and $CVP and If BSC or Binance supports BSCDEFI portfolio by @PowerPoolCVP $CVP is bound to moon,eth,othertweets,long term bullish eth cvp bsc binance support bscdefi portfolio cvp bound moon
145873,2021-08-06,1,1.0,1,1,"🚨Inaugural NUMI Sets 🚨 ⤵️ \n\n1⃣ The OGs: 1 $BTC &amp; 1 $ETH #NUMI \nEarn a limited 1 of 1 #OG #Satoshi / @VitalikButerin #NUMI\n2⃣ ""I Like Big #DEXs &amp; I Cannot Lie"": 1 $UNI, $CAKE, #1INCH, $SUSHI\nFirst 10 to complete earn the #BigDEX #NUMI\n\nContinued.. ⤵️⤵️",sushi,othertweets,inaugural numi set ogs btc eth numi earn limited og satoshi numi like big dexs not lie uni cake 1inch sushi first 10 complete earn bigdex numi continued
145872,2021-08-06,0,0.0,0,0,"I'm ready to get invited after the upgrade of @JoinKlub, adoption is getting near.\n\nBuying more $XKI is one of the best thing to do while waiting and stake it with over 30% APY. 😍\n\n#CeFi #Klub $SUSHI https://t.co/S6Kq8D51vj",sushi,othertweets,ready get invited upgrade adoption getting near buying xki one best thing waiting stake 30 annual percentage yield cefi klub sushi
145871,2021-08-06,0,0.0,0,0,$theta juge profits 💸💸💸\nJoin us on telegram:\nhttps://t.co/qiPeI3rPk1\n\n$theta $tko $zil $tru $atom $coti $sand $axs $pond $twt $matic $luna $alice $doge $sushi $sol https://t.co/OB2bcxvXy1,sushi,othertweets,theta juge profit join u telegram theta tko zil tru atom coti sand ax pond twt matic luna alice doge sushi sol
145870,2021-08-06,1,1.0,1,1,"$SUSHI\nFirst test of the previous resistence now support done, price already up 3% from the test, bullish engulfing is forming on 4H, a close above 9.3 and I don't think 9.6 will cap the price again https://t.co/DXpEPGA6ZK",sushi,othertweets,sushi first test previous resistence now support done price already up test bullish engulfing forming 4h close above 93 not_ think 96 cap price
145869,2021-08-06,0,0.0,0,0,"[Spot] By 2021-08-05 13:00 GMT, SUSHI net cap flow of $27M with a price change of 0.8% in last 24 hrs. Free Alert: https://t.co/10DW3v7XKs #cryptocurrency #CryptoCapFlow $SUSHI https://t.co/OXvL5CDGLs",sushi,othertweets,spot 20210805 1300 gmt sushi net cap flow 27m price change 08 last 24 hr free alert cryptocurrency cryptocapflow sushi
145868,2021-08-06,1,1.0,1,1,$SUSHI $DOT $MKR $LUNA $AAVE $ADA $NEO \n\nALLLLLL looking good right now! \n\n$BTC BEHAVE YOU BIATCH!!\n\n#Crypto #cryptotrading,sushi,othertweets,sushi dot mkr luna aave ada neo allllll looking good right now btc behave biatch crypto cryptotrading


In [274]:
df_2.to_csv('BetterCleaned.csv', index=False)